## Escolha do melhor plano para os usuários da Megaline

Criaremos um modelo que utilize dados comportamentais de usuários que já mudaram para os novos planos (Smart ou Ultra) e, com base nesses dados, seja capaz de selecionar o plano mais adequado para outros usuários. O objetivo é desenvolver um modelo com uma precisão mínima de 75%. Para isso, testaremos três classificadores: Árvore de Decisão, Floresta Aleatória e Regressão Logística.

## Sumário
1. [Iniciação](#inic)

2. [Separando os Grupos de Treinamento, Validação e Teste](#stvt)

3. [Teste de Modelos](#tm)

    A. [Árvore de Decisão](#ad)

    B. [Floresta Aleatória](#fa)

    C. [Regressão Logística](#rl)
    
4. [Verificação de Qualidade com o Conjunto de Teste](#vqct)

5. [Verificação de Integridade](#vi)

6. [Conclusão](#concl)

## Iniciação <a id="inic"></a>

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
try:
    df = pd.read_csv('users_behavior.csv')
except:
    df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


Notamos que não há valores ausentes, porém o tipo de valores das colunas `calls` e `messages` estão incorretos visto que não é possível ter meia ligação ou mensagem.

In [5]:
df['calls'] = df['calls'].astype('int16')
df['messages'] = df['messages'].astype('int16')
df['is_ultra'] = df['is_ultra'].astype('int16')
#os valores são convertidos para int16 para diminuir o tamanho, visto que são valores pequenos

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   int16  
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   int16  
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int16  
dtypes: float64(2), int16(3)
memory usage: 69.2 KB


,calls,minutes,messages,mb_used,is_ultra
0,40,311.90,83,19915.42,0
1,85,516.75,56,22696.96,0
2,77,467.66,86,21060.45,0
3,106,745.53,81,8437.39,1
4,66,418.74,1,14502.75,0


## Separando os Grupos de Treinamento, Validação e Teste <a id="stvt"></a>

Em seguida faremos uso da função `train_test_split()`. Essa função divide um conjunto de dados em duas partes distintas. Entretanto, como necessitamos de três conjuntos de dados distintos, executamos essa divisão duas vezes. Além disso, é importante destacar que as proporções das partes originais do conjunto de dados devem ser distribuídas como 60%, 20%, e 20%, garantindo assim a formação adequada dos conjuntos de treinamento, validação e teste.

In [6]:
df_train, df2 = train_test_split(df, test_size=0.4, random_state=12345)
#divide o dataframe tendo o df_train(60%) e df2(40%)

df_valid, df_test = train_test_split(df2, test_size=0.5, random_state=12345)
#divide o dataframe 2 tendo df_valid(50% do df2, portanto 20% do df) e df_test(50% do df2, portanto 20% do df)

In [7]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1928 entries, 3027 to 482
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     1928 non-null   int16  
 1   minutes   1928 non-null   float64
 2   messages  1928 non-null   int16  
 3   mb_used   1928 non-null   float64
 4   is_ultra  1928 non-null   int16  
dtypes: float64(2), int16(3)
memory usage: 56.5 KB


In [8]:
df_valid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 643 entries, 1386 to 3197
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     643 non-null    int16  
 1   minutes   643 non-null    float64
 2   messages  643 non-null    int16  
 3   mb_used   643 non-null    float64
 4   is_ultra  643 non-null    int16  
dtypes: float64(2), int16(3)
memory usage: 18.8 KB


In [9]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 643 entries, 160 to 2313
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     643 non-null    int16  
 1   minutes   643 non-null    float64
 2   messages  643 non-null    int16  
 3   mb_used   643 non-null    float64
 4   is_ultra  643 non-null    int16  
dtypes: float64(2), int16(3)
memory usage: 18.8 KB


A divisão foi bem sucedida. Neste momento, é necessário estabelecer as características e o target para cada conjunto de dados. Para as características, utilizaremos todo o dataframe, porém excluiremos a coluna alvo. A mesma será composta pela coluna "is_ultra".

In [10]:
train_features = df_train.drop('is_ultra', axis=1)
train_target = df_train['is_ultra']

valid_features = df_valid.drop('is_ultra', axis=1)
valid_target = df_valid['is_ultra']

test_features = df_test.drop('is_ultra', axis=1)
test_target = df_test['is_ultra']

Após definir os conjuntos de características(`features`) e alvo(`target`) testaremos os modelos.

## Teste de Modelos <a id="tm"></a>

Como mencionado anteriormente, avaliaremos a eficácia dos modelos de classificação Árvore de Decisão, Floresta Aleatória e Regressão Logística. Para isso, vamos treinar esses modelos utilizando o conjunto de treinamento e a função `fit()`, e testá-los utilizando o conjunto de validação. A avaliação dos modelos será realizada por meio da comparação entre as previsões realizadas utilizando as características do conjunto de validação, por meio do método `predict()`, e os resultados reais desse conjunto. Além disso, vamos realizar ajustes nos hiperparâmetros dos modelos para obter uma pontuação de acurácia mais alta. Esse resultado será utilizado para escolher o modelo mais eficaz.

### Árvore de Decisão <a id="ad"></a>

Utilizaremos a função `DecisionTreeClassifier()`. Em seguida, definiremos dois hiperparâmetros: `random_state` e `max_depth`. O valor de `random_state` deve ser o mesmo todas as vezes, portanto, será atribuído um valor fixo de `12345`. O hiperparâmetro `max_depth` será o que iremos ajustar, e para isso, vamos realizar um loop em uma série de valores para `max_depth` (no intervalo de 1 a 10) e obter a pontuação de acurácia correspondente a cada valor.

In [11]:
best_accuracy = 0
best_depth = 0
for depth in range(1, 11): 
    tree_model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    tree_model.fit(train_features, train_target)
    tree_valid_pred = tree_model.predict(valid_features)
    accuracy = accuracy_score(valid_target, tree_valid_pred)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_depth = depth
print('Melhor modelo possui max depth =', depth, 'e accuracy =', best_accuracy)

Melhor modelo possui max depth = 10 e accuracy = 0.7853810264385692


Considerando os resultados obtidos, o modelo de Árvore de Decisão mais eficiente é aquele que possui `max_depth` igual a 10, pois apresenta a maior pontuação de acurácia, `78,53%`.

### Floresta Aleatória <a id="fa"></a>

Nesta secção utilizaremos a função `RandomForestClassifier()`. O hiperparâmetro `random_state` deve permanecer o mesmo que antes. Os hiperparâmetros que ajustaremos são `max_depth` e `n_estimators`. Para isso, criaremos inicialmente uma lista vazia. Em seguida, criaremos um loop que percorre os valores de `max_depth` e, dentro desse loop, percorre os valores de `n_estimators`. Utilizaremos esse loop para criar modelos com diferentes combinações de valores para `max_depth` e `n_estimators` que iremos armazenar na lista, a partir da qual escolheremos o modelo com a maior pontuação de acurácia.

In [12]:
rf = [] #rf de RandomForest -> Floresta Aleatória
for depth in range(1, 11):
    for estimators in range(10, 40, 90):
        rf_model = RandomForestClassifier(random_state=12345, max_depth=depth, n_estimators=estimators)
        rf_model.fit(train_features, train_target)
        rf.append(rf_model)
print(max(rf, key=lambda rf_model: accuracy_score(rf_model.predict(valid_features), valid_target)))

RandomForestClassifier(max_depth=8, n_estimators=40, random_state=12345)


De acordo com os resultados, o modelo mais eficiente de classificação floresta aleatória é aquele que possui `max_depth` igual a 8 e `n_estimators` igual a 40. Para esse modelo, utilizaremos o nome `best_rf`. Agora, iremos verificar a sua pontuação de acurácia.

In [13]:
best_rf = RandomForestClassifier(random_state=12345, max_depth=8, n_estimators=40)
best_rf.fit(train_features, train_target)
best_rf_pred = best_rf.predict(valid_features)
print(accuracy_score(valid_target, best_rf_pred))

0.8087091757387247


O melhor modelo obteve uma acuráica de quase `81%` com a classificação por floresta aleatória.

### Regressão Logística <a id="rl"></a>

Aplicaremos a função `LogisticRegression()`. É importante manter o mesmo valor para o hiperparâmetro `random_state`. No entanto, os hiperparâmetros `max_depth` e `n_estimators` não são relevantes neste caso. O que precisamos definir é o `solver`, que será `liblinear`.

In [14]:
#lr de Logistic Regression -> Regressão Logística
lr_model = LogisticRegression(random_state=12345, solver='liblinear')
lr_model.fit(train_features, train_target)
lr_valid_pred = lr_model.predict(valid_features)
print('Acurácia da Regressão Logística =', accuracy_score(valid_target, lr_valid_pred))

Acurácia da Regressão Logística = 0.7573872472783826


A acurária do modelo de regressão logística tem aproximadamente `76%`

Concluímos, portanto, que o modelo mais eficiente é o `Random Forest` com `max_depth=8` e `n_estimators=40` (pontuação de acurácia de `81%`). Em segundo lugar, o modelo `Decision Tree` com `max_depth=3` (pontuação de acurácia de `78,5%`). Por último, o modelo `Logistic Regression` (pontuação de acurácia de `76%`).

## Verificação de Qualidade com o Conjunto de Teste <a id="vqct"></a>

Para aplicar o melhor modelo no conjunto de teste, é necessário antes re-treiná-lo usando tanto o conjunto de treinamento quanto o conjunto de validação combinados. Para isso, podemos usar a função `pd.concat` que recebe uma lista dos conjuntos envolvidos como argumento e definir o parâmetro `axis=0` para empilhamento vertical.

In [15]:
train_final = pd.concat([df_train, df_valid], axis=0)
train_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2571 entries, 3027 to 3197
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     2571 non-null   int16  
 1   minutes   2571 non-null   float64
 2   messages  2571 non-null   int16  
 3   mb_used   2571 non-null   float64
 4   is_ultra  2571 non-null   int16  
dtypes: float64(2), int16(3)
memory usage: 75.3 KB


Treinaremos usando as novas features e target, fazer previsões usando as features do conjunto de teste e obter uma pontuação de precisão. Para isso usaremos o melhor modelo, 'best_rf'.

In [16]:
train_final_features = train_final.drop('is_ultra', axis=1)
train_final_target = train_final['is_ultra']

best_rf.fit(train_final_features, train_final_target)
best_rf_pred = best_rf.predict(test_features)
print(accuracy_score(best_rf_pred, test_target))

0.7993779160186625


Obtivemos uma pontuação de precisão de cerca de `80%`, que está acima do limite de `75%` para o projeto.

## Verificação de Integridade <a id="vi"></a>

Para verificar se o modelo é coerente, é primordial compará-lo com um cenário aleatório. Podemos fazer isso obtendo a pontuação de precisão se nossas previsões fossem basicamente colocar um valor constante para o `target` durante toda a análise. Para isso, primeiro precisamos ver quantos clientes `Smart` e `Ultra` temos no conjunto de teste.

In [17]:
smart_target = (test_target == 0)
ultra_target = (test_target == 1)

print('Número de clientes Smart:', smart_target.sum())
print('Número de clientes Ultra:', ultra_target.sum())

Número de clientes Smart: 440
Número de clientes Ultra: 203


Considerando que temos um número muito maior de clientes `Smart`, usaremos eles como exemplo. Para verificar a eficácia do modelo, realizaremos um teste de integridade comparando-o a uma escolha aleatória. Vamos supor que temos um classificador aleatório que fez previsões somente para a classe 0 (plano `Smart`) em todo o conjunto de teste. Qual seria a pontuação de precisão obtida por esse classificador aleatório?

In [18]:
smart_chance = smart_target.sum() / len(test_target)
print('Acurácia do classificador aleatório Smart:', smart_chance)

Acurácia do classificador aleatório Smart: 0.6842923794712286


O classificador aleatório teria uma pontuação de precisão de `68,4%`, que é inferior aos `80%` que o classificador de floresta aleatória obteve. Então nosso modelo passa na verificação de integridade.

## Conclusão <a id="concl"></a>

Dividimos os dados em conjuntos de treinamento, validação e teste. Testamos vários modelos e constatamos que o `RandomForestClassifier` foi o melhor, obtendo uma precisão de `80%` no conjunto de validação e `79%` no conjunto de teste, além de ter passado no teste de verificação.